<a href="https://colab.research.google.com/github/Chandusatyasai/TECHNEST/blob/TECHNEST/Model_evaluation_%26_tuning2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets rouge-score sacrebleu nltk


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.2 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=001b2e482139655261fa93b3f47d23b0511ff40e0267a49a3b76cb0224c508d3
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [ ]:
import pandas as pd
from datasets import Dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
data = {
    "text": [
        "Artificial Intelligence is transforming various industries by automating complex tasks.",
        "Machine Learning helps in data analysis and building predictive models.",
        "Natural Language Processing enables computers to understand human language.",
        "Neural networks are a core component of deep learning algorithms."
    ],
    "summary": [
        "AI automates complex tasks in industries.",
        "ML helps analyze data and build predictions.",
        "NLP lets computers understand language.",
        "Neural networks power deep learning."
    ]
}

df = pd.DataFrame(data)
dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.3)
dataset = dataset.rename_column("text", "input_text")
dataset = dataset.rename_column("summary", "target_text")
dataset = dataset.map(lambda x: {"input_text": "summarize: " + x["input_text"]})


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [ ]:
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def tokenize(batch):
    inputs = tokenizer(batch["input_text"], padding="max_length", truncation=True, max_length=512)
    targets = tokenizer(batch["target_text"], padding="max_length", truncation=True, max_length=128)
    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized_dataset = dataset.map(tokenize, batched=True, remove_columns=dataset["train"].column_names)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    predict_with_generate=True,
    logging_dir="./logs"
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

<ipython-input-6-a0310e97fa94>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
rouge = load_metric("rouge")
bleu = load_metric("sacrebleu")

def evaluate(index=0):
    input_text = dataset["test"][index]["input_text"]
    reference = dataset["test"][index]["target_text"]

    # Generate prediction
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids
    output_ids = model.generate(input_ids, max_length=128)[0]
    prediction = tokenizer.decode(output_ids, skip_special_tokens=True)

    print("\nINPUT:\n", input_text)
    print("\nREFERENCE:\n", reference)
    print("\nPREDICTION:\n", prediction)

    # Scores
    rouge_score = rouge.compute(predictions=[prediction], references=[reference])
    bleu_score = bleu.compute(predictions=[prediction], references=[[reference]])

    print("\nROUGE SCORE:", rouge_score["rougeL"].mid.fmeasure)
    print("BLEU SCORE:", bleu_score["score"])

evaluate(0)


<ipython-input-8-ffb8d8b9e50f>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")



INPUT:
 summarize: Neural networks are a core component of deep learning algorithms.

REFERENCE:
 Neural networks power deep learning.

PREDICTION:
 neural networks are a component of deep learning algorithms.

ROUGE SCORE: 0.5714285714285714
BLEU SCORE: 9.980099403873663
